# Prepare Epoch Data

### This script's goal is to pre-create as many similarity matrices as possible for training.
### This is because having all w2v data in memory is extremely heavy (~12 GB in RAM)
###### Thank god we had a strong GPU

In [21]:
import dataset_loader as dl
import pacrr
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
import gc
import time
import os

In [2]:
lq = 10
ld = 20
lf = 32
lg = 3
k = 3
lr = 0.01

raw_data_f = r"C:\Users\matan\Desktop\IR_Proj\data\anfL6.json"
token_data_f = "./tokenized_dataset.pickle"
train_data_dir = r"D:\IRfiles\tdata"

In [3]:
# get w2v data.
# questions is array of questions in w2v
# answers is array of length |questions|. each element is an array that contains all answers of each question. (in w2v)

ids, questions, answers, _ = dl.get_dataset_tokens_loaded(raw_data_f, token_data_f)
print("Prepping w2v dataset")
questions, answers = dl.get_w2v_dataset(questions, answers)
print("Fixing w2v length")
questions, answers = dl.fix_length_w2v(questions, answers, lq=lq, ld=ld)
print("done")

Prepping w2v dataset
Fixing w2v length
done


In [16]:
# Garbage collection. w2v is hurting my RAM :( 
gc.collect()


186

In [37]:
# for each question in questions[start : start+dlen] prepare training data
# this means sample a relevent and non-relevent document, and prepare relevent and non-relevent similarity matrices.
def get_train_data(start, dlen, max=None):
    i = start
    
    if max is None:
        max = len(questions)
    
    dp = []
    dn = []
    iterated = 0
    while iterated < dlen:
        if i >= max:
            start = 0
        if len(answers[i]) == 0:
            i+=1
            iterated += 1
            continue
        ps = random.choice(answers[i])
        foundn = False
        while not foundn:
            nsi = random.choice(range(len(questions)-1))
            if nsi != i and len(answers[nsi])>0:
                foundn = True
                ns = random.choice(answers[nsi])
        
        dp.append(dl.get_cosine_mat(questions[i], np.array(ps, dtype=np.float32)))
        dn.append(dl.get_cosine_mat(questions[i], np.array(ns, dtype=np.float32)))
        i += 1
        iterated += 1
        if iterated % 30000 == 0:
            print(iterated)
        
    return np.array(dp), np.array(dn)

In [25]:
# counter for number of epochs
counter = 0

In [39]:
# prepare 300 epochs, save the data to file. This was done over-night
for i in range(300):
    dp, dn = get_train_data(0, len(questions)-2)
    np.save(os.path.join(train_data_dir, r"dp"+str(counter)+".npy"), dp)
    np.save(os.path.join(train_data_dir, r"dp"+str(counter)+".npy"), dn)
    counter += 1
    del dp
    del dn
    gc.collect()

30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
30000
60000
3000

KeyboardInterrupt: 